# Implementación de algoritmo _CVRP_ en simulación generada

In [1408]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from __future__ import print_function
import osmnx as ox
import networkx as nx
import gmaps
import random

### Importar datos de simulación

In [1409]:
n_clientes = 150 #numero de clientes

n_grupo = 10 #numero de simulacion o grupo

directory = f'Simulaciones/{n_clientes} clientes/simulacion_clientes{n_grupo}.csv'

simulacion_clientes = pd.read_csv(directory)
simulacion_clientes.head()


,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3),Coordenadas,Latitud,Longitud
0,San Nicolas,FRES DL LAGO 3ER SAN,66478,AV MEXICO,342.0,0.799117,2,0,0,0,...,0,0,0,0,0,0,0.691320,"(25.7326175, -100.21845745)",25.732617,-100.218457
1,San Nicolas,ARB D STO DOMING SAN,66448,FRESNO,617.0,0.691626,1,0,0,0,...,0,0,0,0,0,0,0.000100,"(25.72523945, -100.2928957)",25.725239,-100.292896
2,Escobedo,MITRAS PONIENTE,66023,SANTOS DEGOLLAD,2007.0,0.935759,3,0,0,0,...,0,0,0,0,0,0,0.246495,"(25.78670159, -100.45779026)",25.786702,-100.457790
3,Escobedo,LAS ENCINAS ESC,66050,PIEDRAS NEGRAS,126.0,0.860676,2,0,0,0,...,0,0,0,0,0,0,0.007377,"(25.8050882, -100.3096367)",25.805088,-100.309637
4,Monterrey,VILLA LOS PINOS,64770,PINO SILVESTRE,2923.0,0.400111,1,0,0,0,...,0,0,0,0,0,0,0.001000,"(25.63611897, -100.28707179)",25.636119,-100.287072


In [1410]:
# Copy the second row
new_row = simulacion_clientes.iloc[1].copy()

# Insert the copied row at the first index
simulacion_clientes.loc[-1] = new_row
simulacion_clientes.index = simulacion_clientes.index + 1
simulacion_clientes = simulacion_clientes.sort_index()
simulacion_clientes.head()

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3),Coordenadas,Latitud,Longitud
0,San Nicolas,ARB D STO DOMING SAN,66448,FRESNO,617.0,0.691626,1,0,0,0,...,0,0,0,0,0,0,0.000100,"(25.72523945, -100.2928957)",25.725239,-100.292896
1,San Nicolas,FRES DL LAGO 3ER SAN,66478,AV MEXICO,342.0,0.799117,2,0,0,0,...,0,0,0,0,0,0,0.691320,"(25.7326175, -100.21845745)",25.732617,-100.218457
2,San Nicolas,ARB D STO DOMING SAN,66448,FRESNO,617.0,0.691626,1,0,0,0,...,0,0,0,0,0,0,0.000100,"(25.72523945, -100.2928957)",25.725239,-100.292896
3,Escobedo,MITRAS PONIENTE,66023,SANTOS DEGOLLAD,2007.0,0.935759,3,0,0,0,...,0,0,0,0,0,0,0.246495,"(25.78670159, -100.45779026)",25.786702,-100.457790
4,Escobedo,LAS ENCINAS ESC,66050,PIEDRAS NEGRAS,126.0,0.860676,2,0,0,0,...,0,0,0,0,0,0,0.007377,"(25.8050882, -100.3096367)",25.805088,-100.309637


In [1411]:
simulacion_clientes["ciudad"][0] = "Monterrey"
simulacion_clientes["Colonia"][0] = "Tecnologico"
simulacion_clientes["codigo postal"][0] = 64849
simulacion_clientes["Calle"][0] = "Eugenio Garza Sada"
simulacion_clientes["numero de casa"][0] = 0
simulacion_clientes["Aleatorio"][0] = 0
simulacion_clientes["Numero de pedidos"][0] = 0
simulacion_clientes["Volumen por pedidos (m^3)"][0] = 0
simulacion_clientes["Coordenadas"][0] = (25.651698, -100.289513)
simulacion_clientes["Latitud"][0] = 25.651698
simulacion_clientes["Longitud"][0] = -100.289513


C:\Users\Ruben\AppData\Local\Temp\ipykernel_1848\843876464.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simulacion_clientes["ciudad"][0] = "Monterrey"
C:\Users\Ruben\AppData\Local\Temp\ipykernel_1848\843876464.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simulacion_clientes["Colonia"][0] = "Tecnologico"
C:\Users\Ruben\AppData\Local\Temp\ipykernel_1848\843876464.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  simulacio

In [1412]:
simulacion_clientes.head()

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3),Coordenadas,Latitud,Longitud
0,Monterrey,Tecnologico,64849,Eugenio Garza Sada,0.0,0.000000,0,0,0,0,...,0,0,0,0,0,0,0.000000,"(25.651698, -100.289513)",25.651698,-100.289513
1,San Nicolas,FRES DL LAGO 3ER SAN,66478,AV MEXICO,342.0,0.799117,2,0,0,0,...,0,0,0,0,0,0,0.691320,"(25.7326175, -100.21845745)",25.732617,-100.218457
2,San Nicolas,ARB D STO DOMING SAN,66448,FRESNO,617.0,0.691626,1,0,0,0,...,0,0,0,0,0,0,0.000100,"(25.72523945, -100.2928957)",25.725239,-100.292896
3,Escobedo,MITRAS PONIENTE,66023,SANTOS DEGOLLAD,2007.0,0.935759,3,0,0,0,...,0,0,0,0,0,0,0.246495,"(25.78670159, -100.45779026)",25.786702,-100.457790
4,Escobedo,LAS ENCINAS ESC,66050,PIEDRAS NEGRAS,126.0,0.860676,2,0,0,0,...,0,0,0,0,0,0,0.007377,"(25.8050882, -100.3096367)",25.805088,-100.309637


In [1413]:
nuevas_coordenadas = []
for i in range(simulacion_clientes.shape[0]):
    coord =  (simulacion_clientes.iloc[i,4770],simulacion_clientes.iloc[i,4771])
    nuevas_coordenadas.append(coord)

simulacion_clientes["Coordenadas"] = nuevas_coordenadas

In [1414]:
simulacion_clientes["Coordenadas"][0]

(25.651698, -100.289513)

In [1415]:
#Ejemplo
distancia = str(geodesic(simulacion_clientes.iloc[0,4769], simulacion_clientes.iloc[1,4769]).km)
print("Distancia entre dos puntos",distancia)

Distancia entre dos puntos 11.455710067852495


In [1416]:
simulacion_clientes["Latitud"][1]

25.7326175

In [1417]:
simulacion_clientes["Longitud"][1]

-100.21845745

In [1418]:
simulacion_clientes["Latitud"][2]

25.72523945

In [1419]:
simulacion_clientes["Longitud"][2]

-100.2928957

In [1420]:
simulacion_clientes["Latitud"][1]

25.7326175

In [1421]:
def generar_distancias(clientes):
    matriz_distancias = []

    for i in range(clientes.shape[0]):
        coord1 = clientes.iloc[i,4769]
        distancias =  []

        for j in range(clientes.shape[0]):
            if j != i:
                coord2 = clientes.iloc[j,4769]
                distancia = float(geodesic(coord1, coord2).km)
                distancias.append(distancia)
            else:
                distancias.append(0)

        matriz_distancias.append(distancias)


    return matriz_distancias


In [1422]:
distancias = generar_distancias(simulacion_clientes)

In [1423]:
pd.DataFrame(np.matrix(distancias)).head()

,0,1,2,3,4,5,6,7,8,9,...,141,142,143,144,145,146,147,148,149,150
0,0.000000,11.455710,8.154233,22.558178,17.112719,1.743208,29.129824,11.647175,9.862856,15.572574,...,4.355464,4.053798,10.254146,18.044182,10.509359,9.022845,10.480866,30.941522,7.973889,16.571196
1,11.455710,0.000000,7.514195,24.746313,12.170307,12.717162,18.339124,8.783831,6.813252,11.416260,...,10.866811,7.798223,12.070844,9.465301,5.043393,17.125224,6.872783,33.352367,5.858808,14.346356
2,8.154233,7.514195,0.000000,17.889326,9.003999,9.890340,22.049645,3.552412,1.851915,7.514828,...,4.915750,4.566309,4.677136,10.294890,3.349715,10.130402,2.447296,26.582209,1.742032,9.116793
3,22.558178,24.746313,17.889326,0.000000,14.997308,23.913343,30.676459,15.984661,17.994014,14.584704,...,18.402961,21.246863,13.351837,20.550174,19.703665,14.983335,17.876277,8.693961,19.592360,11.304661
4,17.112719,12.170307,9.003999,14.997308,0.000000,18.855511,16.638332,5.467414,7.528062,1.616181,...,13.333846,13.529309,8.524760,5.552973,7.971880,16.032439,6.963349,22.877583,9.749819,3.835061


In [1424]:
demandas = simulacion_clientes["Volumen por pedidos (m^3)"].tolist()

In [1425]:
# El primer nodo no tiene demanda
demandas[0] = 0

In [1426]:
# Suma de la demanda de todos los clientes(en m^3)
simulacion_clientes["Volumen por pedidos (m^3)"].sum()

31.65946341

In [1427]:
pedidos = simulacion_clientes["Numero de pedidos"].to_list()

In [1428]:
distancias_escaladas = [[int(round(d * 1000)) for d in row] for row in distancias] 

In [1429]:
demandas_escaladas = [int(round(p * 1000)) for p in demandas]

In [1430]:
capacidades = pd.read_excel("Unidades_transporte.xlsx")["Capacidad"].tolist() #capacidades de las unidades

In [1431]:
#Determinar numero de vehiculos

n_vehiculos = 0
demanda_enVolumen =simulacion_clientes["Volumen por pedidos (m^3)"].sum()

for i in range(len(capacidades)):
    if sum(capacidades[:i]) < demanda_enVolumen:
        n_vehiculos += 1
    else:
        continue

print(f'"Numero de vehiculos a utilizar: {n_vehiculos}')

"Numero de vehiculos a utilizar: 4


In [1432]:
capacidades_escaladas = [c * 1000 for c in capacidades][:n_vehiculos]
capacidades_escaladas

[8000, 8000, 8000, 8000]

In [1433]:
orden_delivery = {}

volumenEnCadaLugar = {}

kilometrajeRecorrido_acumul = {}

for i in range(n_vehiculos):
    orden_delivery[i] = []
    volumenEnCadaLugar[i] = []
    kilometrajeRecorrido_acumul[i] = []


In [1434]:
distancias

[[0,
  11.455710067852495,
  8.154233066763341,
  22.55817798114409,
  17.112718540807066,
  1.7432075354298728,
  29.129823848056688,
  11.647174939965243,
  9.86285560475917,
  15.572574196058596,
  11.968455571892443,
  11.950140712291331,
  16.10913207696909,
  12.087348442280181,
  17.46466769845858,
  16.62090019850689,
  12.891826380543437,
  10.699090082103018,
  20.021265527331348,
  9.841644476668248,
  17.855557148902424,
  14.118102403183501,
  27.933439193012333,
  9.750536728011271,
  5.441813271882644,
  5.7399776281459545,
  9.525848655843255,
  10.78998301381538,
  2.0822669978223454,
  6.576614458615312,
  11.37379332853678,
  11.159310438498474,
  13.451084641767922,
  7.937837094504129,
  29.129823848056688,
  10.894514330429665,
  3.297841591151961,
  20.497913619909294,
  7.043444944235304,
  4.093202133714434,
  19.388614787349912,
  2.5847618260092675,
  16.9127630328084,
  17.579452815138616,
  17.749873887693703,
  8.823264755366246,
  7.521149773492016,
  10.

In [1435]:
average_speed = 30

time_perkm =  1/ average_speed


time_matrix = np.matrix(distancias) * time_perkm


time_matrix_scaled = [[int(round(d * 1000)) for d in row] for row in time_matrix.tolist()]

time_matrix_scaled[:10]

[[0,
  382,
  272,
  752,
  570,
  58,
  971,
  388,
  329,
  519,
  399,
  398,
  537,
  403,
  582,
  554,
  430,
  357,
  667,
  328,
  595,
  471,
  931,
  325,
  181,
  191,
  318,
  360,
  69,
  219,
  379,
  372,
  448,
  265,
  971,
  363,
  110,
  683,
  235,
  136,
  646,
  86,
  564,
  586,
  592,
  294,
  251,
  348,
  404,
  350,
  229,
  198,
  312,
  357,
  380,
  520,
  506,
  63,
  534,
  348,
  568,
  550,
  434,
  730,
  373,
  629,
  370,
  489,
  562,
  849,
  386,
  58,
  776,
  836,
  281,
  235,
  619,
  127,
  581,
  332,
  581,
  347,
  177,
  941,
  186,
  402,
  484,
  666,
  979,
  789,
  352,
  351,
  516,
  438,
  343,
  413,
  320,
  123,
  548,
  352,
  131,
  337,
  669,
  522,
  556,
  748,
  410,
  393,
  464,
  157,
  599,
  505,
  63,
  306,
  442,
  360,
  415,
  356,
  241,
  482,
  663,
  317,
  322,
  467,
  646,
  317,
  357,
  410,
  382,
  310,
  822,
  46,
  231,
  310,
  133,
  133,
  483,
  522,
  328,
  244,
  673,
  145,
  135,
  342,
 

In [1436]:
time_windows = []
for i in range(len(time_matrix)):
    time_windows.append((random.randint(9,18),random.randint(9,18)))

### Algoritmo de busqueda de rutas (_CVRP_)

In [1437]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distancias_escaladas
    data["demands"] = demandas_escaladas
    data["vehicle_capacities"] = capacidades_escaladas
    data["num_vehicles"] = n_vehiculos
    data["depot"] = 0         
    
    return data

In [1438]:
def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        ordenDeLlegada = []
        volumenDeCadaLugar = []
        km_recorrido_acumul = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load/ 1000}) -> "
            ordenDeLlegada.append(node_index)
            volumenDeCadaLugar.append(route_load / 1000)
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
            km_recorrido_acumul.append(route_distance/ 1000)
        plan_output += f" {manager.IndexToNode(index)} Load({route_load / 1000})\n"
        plan_output += f"Distance of the route: {route_distance / 1000}km\n"
        plan_output += f"Load of the route: {route_load / 1000}\n"
        orden_delivery[vehicle_id] = ordenDeLlegada
        volumenEnCadaLugar[vehicle_id] = volumenDeCadaLugar
        kilometrajeRecorrido_acumul[vehicle_id] = km_recorrido_acumul
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance / 1000}km")
    print(f"Total load of all routes: {total_load / 1000}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

Objective: 352890
Route for vehicle 0:
 0 Load(0.0) ->  82 Load(0.005) ->  25 Load(0.026) ->  149 Load(0.028) ->  45 Load(0.162) ->  133 Load(0.385) ->  79 Load(0.388) ->  101 Load(0.389) ->  145 Load(0.497) ->  53 Load(0.503) ->  117 Load(0.594) ->  10 Load(0.595) ->  95 Load(0.784) ->  114 Load(0.786) ->  32 Load(0.793) ->  106 Load(0.8) ->  62 Load(0.853) ->  67 Load(1.368) ->  78 Load(1.539) ->  144 Load(1.55) ->  124 Load(2.19) ->  6 Load(2.191) ->  34 Load(2.202) ->  88 Load(2.203) ->  120 Load(5.254) ->  15 Load(5.256) ->  58 Load(5.558) ->  93 Load(5.559) ->  13 Load(6.106) ->  11 Load(6.169) ->  35 Load(6.209) ->  128 Load(6.238) ->  1 Load(6.929) ->  17 Load(7.744) ->  121 Load(7.764) ->  0 Load(7.764)
Distance of the route: 72.814km
Load of the route: 7.764

Route for vehicle 1:
 0 Load(0.0) ->  100 Load(0.001) ->  24 Load(0.041) ->  125 Load(0.042) ->  30 Load(0.643) ->  59 Load(3.213) ->  126 Load(3.457) ->  116 Load(3.457) ->  21 Load(3.464) ->  92 Load(3.687) ->  104 Loa

In [1439]:
for vehicle_id in volumenEnCadaLugar.keys():
    volumenPorVehiculo = volumenEnCadaLugar[vehicle_id]
    volumenTotalPorVehiculo = volumenPorVehiculo[-1]
    nuevosVolumenes = []
    for i in range(len(volumenPorVehiculo)):
        newVolumen = volumenTotalPorVehiculo - volumenPorVehiculo[i]
        nuevosVolumenes.append(newVolumen)
    
    volumenEnCadaLugar[vehicle_id] =  nuevosVolumenes

In [1440]:
simulacion_clientes.head()

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3),Coordenadas,Latitud,Longitud
0,Monterrey,Tecnologico,64849,Eugenio Garza Sada,0.0,0.000000,0,0,0,0,...,0,0,0,0,0,0,0.000000,"(25.651698, -100.289513)",25.651698,-100.289513
1,San Nicolas,FRES DL LAGO 3ER SAN,66478,AV MEXICO,342.0,0.799117,2,0,0,0,...,0,0,0,0,0,0,0.691320,"(25.7326175, -100.21845745)",25.732617,-100.218457
2,San Nicolas,ARB D STO DOMING SAN,66448,FRESNO,617.0,0.691626,1,0,0,0,...,0,0,0,0,0,0,0.000100,"(25.72523945, -100.2928957)",25.725239,-100.292896
3,Escobedo,MITRAS PONIENTE,66023,SANTOS DEGOLLAD,2007.0,0.935759,3,0,0,0,...,0,0,0,0,0,0,0.246495,"(25.78670159, -100.45779026)",25.786702,-100.457790
4,Escobedo,LAS ENCINAS ESC,66050,PIEDRAS NEGRAS,126.0,0.860676,2,0,0,0,...,0,0,0,0,0,0,0.007377,"(25.8050882, -100.3096367)",25.805088,-100.309637


In [1441]:

# List of indices you want to keep in order
indices_to_keep_v0 = orden_delivery[0]
indices_to_keep_v1 = orden_delivery[1]
indices_to_keep_v2 = orden_delivery[2]
indices_to_keep_v3 = orden_delivery[3]
"""
indices_to_keep_v4 = orden_delivery[4]
indices_to_keep_v5 = orden_delivery[5]
indices_to_keep_v6 = orden_delivery[6]
"""

# Filter the DataFrame using loc and preserve the order of indices
rutasVehiculo0 = simulacion_clientes.loc[indices_to_keep_v0]
rutasVehiculo1 = simulacion_clientes.loc[indices_to_keep_v1]
rutasVehiculo2 = simulacion_clientes.loc[indices_to_keep_v2]
rutasVehiculo3 = simulacion_clientes.loc[indices_to_keep_v3]
"""
rutasVehiculo4 = simulacion_clientes.loc[indices_to_keep_v4]
rutasVehiculo5 = simulacion_clientes.loc[indices_to_keep_v5]
rutasVehiculo6 = simulacion_clientes.loc[indices_to_keep_v6]
"""



'\nrutasVehiculo4 = simulacion_clientes.loc[indices_to_keep_v4]\nrutasVehiculo5 = simulacion_clientes.loc[indices_to_keep_v5]\nrutasVehiculo6 = simulacion_clientes.loc[indices_to_keep_v6]\n'

In [1442]:
#Vehiculo 0

rutasVehiculo0["Kilometraje Recorrido en ese punto"] = kilometrajeRecorrido_acumul[0]
rutasVehiculo0["Volumen del camion al llegar en el lugar"] = volumenEnCadaLugar[0]


#Vehiculo 1

rutasVehiculo1["Kilometraje Recorrido en ese punto"] = kilometrajeRecorrido_acumul[1]
rutasVehiculo1["Volumen del camion al llegar en el lugar"] = volumenEnCadaLugar[1]

rutasVehiculo2["Kilometraje Recorrido en ese punto"] = kilometrajeRecorrido_acumul[2]
rutasVehiculo2["Volumen del camion al llegar en el lugar"] = volumenEnCadaLugar[2]

#Vehiculo 3

rutasVehiculo3["Kilometraje Recorrido en ese punto"] = kilometrajeRecorrido_acumul[3]
rutasVehiculo3["Volumen del camion al llegar en el lugar"] = volumenEnCadaLugar[3]
"""



#Vehiculo 4

rutasVehiculo4["Kilometraje Recorrido en ese punto"] = kilometrajeRecorrido_acumul[4]
rutasVehiculo4["Volumen del camion al llegar en el lugar"] = volumenEnCadaLugar[4]

#Vehiculo 5

rutasVehiculo5["Kilometraje Recorrido en ese punto"] = kilometrajeRecorrido_acumul[5]
rutasVehiculo5["Volumen del camion al llegar en el lugar"] = volumenEnCadaLugar[5]


#Vehiculo 6

rutasVehiculo6["Kilometraje Recorrido en ese punto"] = kilometrajeRecorrido_acumul[6]
rutasVehiculo6["Volumen del camion al llegar en el lugar"] = volumenEnCadaLugar[6]
"""
#Vehiculo 2


'\n\n\n\n#Vehiculo 4\n\nrutasVehiculo4["Kilometraje Recorrido en ese punto"] = kilometrajeRecorrido_acumul[4]\nrutasVehiculo4["Volumen del camion al llegar en el lugar"] = volumenEnCadaLugar[4]\n\n#Vehiculo 5\n\nrutasVehiculo5["Kilometraje Recorrido en ese punto"] = kilometrajeRecorrido_acumul[5]\nrutasVehiculo5["Volumen del camion al llegar en el lugar"] = volumenEnCadaLugar[5]\n\n\n#Vehiculo 6\n\nrutasVehiculo6["Kilometraje Recorrido en ese punto"] = kilometrajeRecorrido_acumul[6]\nrutasVehiculo6["Volumen del camion al llegar en el lugar"] = volumenEnCadaLugar[6]\n'

In [1443]:
rutasVehiculo0

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52984,52993,53014,53475,Volumen por pedidos (m^3),Coordenadas,Latitud,Longitud,Kilometraje Recorrido en ese punto,Volumen del camion al llegar en el lugar
0,Monterrey,Tecnologico,64849,Eugenio Garza Sada,0.0,0.000000,0,0,0,0,...,0,0,0,0,0.000000,"(25.651698, -100.289513)",25.651698,-100.289513,5.295,7.764
82,Monterrey,MODERNA,64530,GARDENIA,3337.0,0.858335,2,0,0,0,...,0,0,0,0,0.005049,"(25.6993077, -100.2848439)",25.699308,-100.284844,5.899,7.759
25,Monterrey,MODERNA,64530,PRIVADA INDEPEN,3022.0,0.848674,2,0,0,0,...,0,0,0,0,0.020919,"(25.7035053, -100.28868245)",25.703505,-100.288682,8.378,7.738
149,San Nicolas,JARD DE NOGALAR SAN,66486,DA VINCI,466.0,0.410769,1,0,0,0,...,0,0,0,0,0.001617,"(25.72259225, -100.275784)",25.722592,-100.275784,9.267,7.736
45,San Nicolas,CARMEN ROMANO,66444,FINLANDESES,417.0,0.695386,1,0,0,0,...,0,0,0,0,0.134097,"(25.7296749, -100.2716205)",25.729675,-100.271620,10.014,7.602
133,San Nicolas,PEDREGAL STO DOM SAN,66444,JUAN SARABIA,149.0,0.730668,2,0,0,0,...,0,0,0,0,0.223000,"(25.7325517, -100.2648893)",25.732552,-100.264889,10.809,7.379
79,San Nicolas,LAS PUENTES III SAN,66460,VOLCAN SANTA MA,1138.0,0.551800,1,0,0,0,...,0,0,0,0,0.002554,"(25.7392742, -100.2676492)",25.739274,-100.267649,11.145,7.376
101,San Nicolas,BALC D STO DOMIN SAN,66446,C SAN PABLO,1331.0,0.556951,1,0,0,0,...,0,0,0,0,0.000560,"(25.7401524, -100.26444095)",25.740152,-100.264441,11.677,7.375
145,San Nicolas,A#O DE JUAREZ SAN,66460,FRANCISCO I MAD,437.0,0.977336,4,0,0,0,...,0,0,0,0,0.107796,"(25.744348, -100.267024)",25.744348,-100.267024,12.845,7.267
53,San Nicolas,UNIDAD LABORAL SAN,66440,ALBERTO TORRES,508.0,0.634713,1,0,0,0,...,0,0,0,0,0.006420,"(25.7432149, -100.2554472)",25.743215,-100.255447,12.995,7.261


### Exportar las rutas

In [1444]:
directory_v0 = f'rutas/{n_clientes} clientes/Simulacion{n_grupo}/rutasVehiculo0.csv'
directory_v1 = f'rutas/{n_clientes} clientes/Simulacion{n_grupo}/rutasVehiculo1.csv'
directory_v2 = f'rutas/{n_clientes} clientes/Simulacion{n_grupo}/rutasVehiculo2.csv'
directory_v3 = f'rutas/{n_clientes} clientes/Simulacion{n_grupo}/rutasVehiculo3.csv'


rutasVehiculo0.to_csv(directory_v0)
rutasVehiculo1.to_csv(directory_v1)
rutasVehiculo2.to_csv(directory_v2)
rutasVehiculo3.to_csv(directory_v3)

"""
rutasVehiculo3.to_csv("rutas_vehiculo3.csv")
rutasVehiculo4.to_csv("rutas_vehiculo4.csv")
rutasVehiculo5.to_csv("rutas_vehiculo5.csv")
rutasVehiculo6.to_csv("rutas_vehiculo6.csv")
"""

'\nrutasVehiculo3.to_csv("rutas_vehiculo3.csv")\nrutasVehiculo4.to_csv("rutas_vehiculo4.csv")\nrutasVehiculo5.to_csv("rutas_vehiculo5.csv")\nrutasVehiculo6.to_csv("rutas_vehiculo6.csv")\n'